There's been a Murder in SQL City!!

The detective gave us the crime scene report, but we somehow lost it. We vaguely remember that the crime was a ​**murder**​ that occurred sometime on **​15 Jan 2018​** and that it took place in **​SQL City**​. 

Let's use our Murder Mystery database to try to solve it!!!

In [71]:
# import libraries
import pandas as pd
import sqlite3 as sql

In [72]:
# setting up connection to the database
con = sql.connect('database/sql-murder-mystery.db')

Let's search through the crime scene reports using all information available:
* what:   murder
* when:   15 January 2018
* where:  SQL City 

In [73]:
# Checking crime scene reports
query_1 = '''
SELECT *
FROM 
    crime_scene_report
WHERE 
    city = "SQL City" 
    AND date = "20180115" 
    AND type = "murder";
'''

#seting the dataframe width to max
pd.set_option('display.max_colwidth', None)

#running our query
SQL_city_crime = pd.read_sql_query(query_1, con)
SQL_city_crime

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


Based on the crime note, there were two witnesses. Let’s look at their interview statements.

    Witness 1




In [74]:
witness_1 = '''
SELECT 
    p.id, 
    p.name, 
    p.address_street_name, 
    p.address_number, 
    i.transcript
FROM 
    person AS p
INNER JOIN 
    interview AS i ON p.id = i.person_id
WHERE 
    p.address_street_name == "Northwestern Dr" 
    AND address_number = (
        SELECT MAX(address_number) 
        FROM 
            person 
        WHERE 
            address_street_name = "Northwestern Dr");
'''

df = pd.read_sql_query(witness_1, con)
df

,id,name,address_street_name,address_number,transcript
0,14887,Morty Schapiro,Northwestern Dr,4919,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."



    Witness 2

In [75]:
witness_2 = '''
SELECT 
    p.id, 
    p.name, 
    p.address_street_name, 
    p.address_number, 
    i.transcript
FROM 
    person AS p
INNER JOIN 
    interview AS i ON p.id = i.person_id
WHERE 
    p.name LIKE "%Annabel%"
    AND address_street_name = "Franklin Ave";
'''

w2 = pd.read_sql_query(witness_2, con)
w2

,id,name,address_street_name,address_number,transcript
0,16371,Annabel Miller,Franklin Ave,103,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


The second witness saw the murderer, and we'll examine her statement first to identify potential suspects.

In [76]:
suspects_q = '''
WITH ann_gym_times AS (
  SELECT
      ci.check_in_time  AS ann_in,
      ci.check_out_time AS ann_out
  FROM 
      get_fit_now_check_in ci
  INNER JOIN 
      get_fit_now_member mem ON ci.membership_id = mem.id
  WHERE 
      ci.check_in_date = '20180109'
      AND mem.name = 'Annabel Miller'
)
SELECT 
    mem.*, 
    ci.check_in_date, 
    ci.check_in_time, 
    ci.check_out_time
FROM 
    get_fit_now_check_in ci
JOIN 
    get_fit_now_member mem ON ci.membership_id = mem.id
CROSS JOIN 
    ann_gym_times at
WHERE 
    ci.check_in_date = '20180109'
    AND ci.check_out_time >= at.ann_in
    AND ci.check_in_time  <= at.ann_out;
'''

suspects = pd.read_sql_query(suspects_q, con)
suspects

,id,person_id,name,membership_start_date,membership_status,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,20180109,1530,1700
2,90081,16371,Annabel Miller,20160208,gold,20180109,1600,1700


Two other people were present in the gym when our witness (Annabel Miller) was there on 9 January 2018. Let’s examine the first witness’s statement for confirmation of the suspect.

In [77]:
suspect_q = '''
SELECT 
    m.id, 
    p.name, 
    d.age, 
    d.height, 
    d.eye_color, 
    d.hair_color, 
    d.gender, 
    d.plate_number, 
    d.car_make, 
    d.car_model
FROM 
  get_fit_now_member AS m
JOIN 
  person AS p ON m.person_id = p.id
JOIN 
  drivers_license AS d ON p.license_id = d.id
WHERE 
  m.id LIKE '%48Z%'
  AND m.membership_status = 'gold'
  AND d.plate_number LIKE '%H42W%';
'''

suspect = pd.read_sql_query(suspect_q, con)
suspect

,id,name,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,48Z55,Jeremy Bowers,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


Our primary suspect has been identified as Jeremy Bowers.

Our solution has been checked on the official Murder Mystery website [link](https://mystery.knightlab.com/).

Results:

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


It is not the end...


In [78]:
suspect_statment = '''
SELECT 
    p.id, 
    p.name, 
    i.transcript
FROM 
    person AS p
JOIN 
    interview AS i ON p.id = i.person_id
WHERE 
    name = 'Jeremy Bowers';
'''


suspect_s = pd.read_sql_query(suspect_statment, con)
suspect_s

,id,name,transcript
0,67318,Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


As it turned out, he was working for someone. Let's look for the real villain...

In [79]:
villian_search = '''
SELECT
    p.name,
    d.age,
    d.height,
    d.eye_color,
    d.gender,
    d.car_make,
    d.car_model,
    i.annual_income,
    f.event_name,
    COUNT (*) AS event_count,
    f.date
FROM
    person AS p
JOIN
    drivers_license AS d ON p.license_id = d.id
JOIN
    facebook_event_checkin AS f ON p.id = f.person_id
JOIN
    income AS i ON p.ssn = i.ssn
WHERE
    d.gender = 'female'
    AND d.height BETWEEN 65 AND 67
    AND d.hair_color = 'red'
    AND d.car_make = 'Tesla'
    AND d.car_model = 'Model S'
    AND event_name LIKE "SQL Symphony Concert"
    AND date LIKE "2017%";
'''

villian = pd.read_sql_query(villian_search, con)
villian

,name,age,height,eye_color,gender,car_make,car_model,annual_income,event_name,event_count,date
0,Miranda Priestly,68,66,green,female,Tesla,Model S,310000,SQL Symphony Concert,3,20171206


Let's make sure that we have found the right person responsible for the murder!

Our primary villain behind the murder is Miranda Priestly.

Again, our suspect's name has been checked on the official Murder Mystery website [link](https://mystery.knightlab.com/).


Result:
Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!
